In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= "C:/Users/USER02/Documents/Voice2Text.json"

In [9]:
from enum import Enum
import io

from google.cloud import vision
from google.cloud.vision import types
from PIL import Image, ImageDraw
# [END vision_document_text_tutorial_imports]


class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5


def draw_boxes(image, bounds, color):
    """Draw a border around the image using the hints in the vector list."""
    draw = ImageDraw.Draw(image)

    for bound in bounds:
        draw.polygon([
            bound.vertices[0].x, bound.vertices[0].y,
            bound.vertices[1].x, bound.vertices[1].y,
            bound.vertices[2].x, bound.vertices[2].y,
            bound.vertices[3].x, bound.vertices[3].y], None, color)
    return image


def get_document_bounds(image_file, feature):
    # [START vision_document_text_tutorial_detect_bounds]
    """Returns document bounds given an image."""
    client = vision.ImageAnnotatorClient()

    bounds = []

    with io.open(image_file, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    response = client.document_text_detection(image=image)
    document = response.full_text_annotation
    
    print(document)

    # Collect specified feature bounds by enumerating all document features
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if (feature == FeatureType.SYMBOL):
                            bounds.append(symbol.bounding_box)

                    if (feature == FeatureType.WORD):
                        bounds.append(word.bounding_box)

                if (feature == FeatureType.PARA):
                    bounds.append(paragraph.bounding_box)

            if (feature == FeatureType.BLOCK):
                bounds.append(block.bounding_box)

        if (feature == FeatureType.PAGE):
            bounds.append(block.bounding_box)

    # The list `bounds` contains the coordinates of the bounding boxes.
    # [END vision_document_text_tutorial_detect_bounds]
    return bounds


def render_doc_text(filein, fileout):
    image = Image.open(filein)
    bounds = get_document_bounds(filein, FeatureType.PAGE)
    draw_boxes(image, bounds, 'blue')
    bounds = get_document_bounds(filein, FeatureType.PARA)
    draw_boxes(image, bounds, 'red')
    bounds = get_document_bounds(filein, FeatureType.WORD)
    draw_boxes(image, bounds, 'yellow')

    if fileout is not 0:
        image.save(fileout)
    else:
        image.show()

In [10]:
render_doc_text('resource/mail1.jpg','out1.jpg')

pages {
  property {
    detected_languages {
      language_code: "ja"
      confidence: 0.7099999785423279
    }
  }
  width: 3120
  height: 4160
  blocks {
    bounding_box {
      vertices {
        x: 442
        y: 1182
      }
      vertices {
        x: 1824
        y: 1209
      }
      vertices {
        x: 1821
        y: 1338
      }
      vertices {
        x: 439
        y: 1311
      }
    }
    paragraphs {
      bounding_box {
        vertices {
          x: 442
          y: 1182
        }
        vertices {
          x: 1824
          y: 1209
        }
        vertices {
          x: 1821
          y: 1338
        }
        vertices {
          x: 439
          y: 1311
        }
      }
      words {
        property {
          detected_languages {
            language_code: "ja"
          }
        }
        bounding_box {
          vertices {
            x: 442
            y: 1182
          }
          vertices {
            x: 1824
            y: 1209
          }
